In [32]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [33]:
dict_tablas = pickle.load(open('dict_tablas', 'rb'))
df_historical_data= pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')


## voy a limpiar los datos del fixture

In [34]:
df_fixture2 = df_fixture

In [35]:
df_fixture2['score'] = df_fixture2['score'].str.replace('[()aet.]', '', regex=True)
df_fixture2

,home,score,away,year
0,Qatar,0–2,Ecuador,2022
1,Senegal,0–2,Netherlands,2022
2,Qatar,1–3,Senegal,2022
3,Netherlands,1–1,Ecuador,2022
4,Ecuador,1–2,Senegal,2022
...,...,...,...,...
59,England,1–2,France,2022
60,Argentina,3–0,Croatia,2022
61,France,2–0,Morocco,2022
62,Croatia,2–1,Morocco,2022


## ------

In [36]:
df_historical_data

,HomeTeam,Awayteam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,Germany,Mexico,2018,0,1,1


### Creacion team strenght

In [37]:
df_historical_data

,HomeTeam,Awayteam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,Germany,Mexico,2018,0,1,1


In [38]:
## Dividir df en df_home y df_away
df_home = df_historical_data[[	'HomeTeam', 'HomeGoals',	'AwayGoals'	]]
df_away = df_historical_data[[	'Awayteam', 'HomeGoals',	'AwayGoals'	]]

In [39]:
## Renombramos columnas
## o podriamos con el inplace = True en vez de volver a llamar el dataframe igual para actualizar
df_home = df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals' : 'GoalsScored',	'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'Awayteam': 'Team', 'HomeGoals' : 'GoalsConceded',	'AwayGoals': 'GoalsScored'})

In [40]:
## concatenemos df home y df away 
df_team_strength = pd.concat([df_home,df_away], ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [41]:
## crear funcion de prediccion
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index :
        #goals scored * goals_conceded
        lamb_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, 'GoalsConceded']
        lamb_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, 'GoalsConceded']
        prob_home, prob_away, prob_draw = 0,0,0

        ## simularia todos los posibles resultados 10 veces aprox
        for x in range(0,11): ## number of goals home team
            for y in range (0,11): ##number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf (y, lamb_away)
                if x == y :
                    prob_draw += p
                elif x>y:
                    prob_home += p
                elif x<y:
                    prob_away += p
        points_home = 3* prob_home + prob_draw
        points_away = 3* prob_away + prob_draw
        return(points_home, points_away)
    
    else:
        ## puede pasar porque hay equipos que nunca han participado en un mundial entonces no tenemos datos historicos para analizar
        return(0,0)


    

## 2.1Ensayemos la funcion

In [42]:
## ensayemos los partidos Argentina—Mexico, England — United Stated, Qatar (H) — Ecuador
predict_points('Argentina', 'Mexico')
predict_points('England', 'United States')
predict_points('Qatar (H)', 'Ecuador')


(0, 0)

## Prediciendo mundial

### 3.1 Fase de grupos

In [43]:
## Dividiendo fixture en grupo , octavos , cuartos, .....
df_fixture_group_48 = df_fixture[2:48].copy()
df_fixture_knockout = df_fixture2[48:56].copy()
df_fixture_quarter = df_fixture2[56:60].copy()
df_fixture_semi = df_fixture2[60:62].copy()
df_fixture_final = df_fixture2[62:].copy()

In [44]:
for group in dict_tablas:
    print(dict_tablas[group]['Team'].values)

['Netherlands' 'Senegal' 'Ecuador' 'Qatar (H)']
['England' 'United States' 'Iran' 'Wales']
['Argentina' 'Poland' 'Mexico' 'Saudi Arabia']
['France' 'Australia' 'Tunisia' 'Denmark']
['Japan' 'Spain' 'Germany' 'Costa Rica']
['Morocco' 'Croatia' 'Belgium' 'Canada']
['Brazil' 'Switzerland' 'Cameroon' 'Serbia']
['Portugal' 'South Korea' 'Uruguay' 'Ghana']


In [69]:
dict_tablas['Grupo C'].loc[dict_tablas['Grupo C']['Team'] == 'Argentina' , 'Pts'] 

0    13.0
Name: Pts, dtype: float64

In [46]:
dict_tablas['Grupo C']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Argentina,3,2,0,1,5,2,+3,6
1,2,Poland,3,1,1,1,2,2,0,4
2,3,Mexico,3,1,1,1,2,3,−1,4
3,4,Saudi Arabia,3,1,0,2,3,5,−2,3


In [47]:
## correr todos los partidos de la fase de grupo y actualizar las tablas de cada grupo
for group in dict_tablas:
    teams_in_group = dict_tablas[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home , points_away = predict_points(home, away)
        dict_tablas[group].loc[dict_tablas[group]['Team'] == home , 'Pts'] += points_home
        dict_tablas[group].loc[dict_tablas[group]['Team'] == away , 'Pts'] += points_away

    dict_tablas[group] = dict_tablas[group].sort_values('Pts', ascending = False).reset_index() ## ordenarlo por puntos
    dict_tablas[group] = dict_tablas[group][['Team', 'Pts']]
    dict_tablas[group] = dict_tablas[group].round(0)



In [48]:
## mostrar tabla actualizada
dict_tablas['Grupo A']

,Team,Pts
0,Netherlands,9.0
1,Senegal,8.0
2,Ecuador,6.0
3,Qatar (H),0.0


### 3.2 Octavos

In [66]:
import pandas as pd

# Crear una lista de listas con los datos de la tabla
datavacio = [[48, "Winners Grupo A", "Match 49", "Runners-up Grupo B", 2022],
          [49, "Winners Grupo C", "Match 50", "Runners-up Grupo D", 2022],
          [50, "Winners Grupo D", "Match 52", "Runners-up Grupo C", 2022],
          [51, "Winners Grupo B", "Match 51", "Runners-up Grupo A", 2022],
          [52, "Winners Grupo E", "Match 53", "Runners-up Grupo F", 2022],
          [53, "Winners Grupo G", "Match 54", "Runners-up Grupo H", 2022],
          [54, "Winners Grupo F", "Match 55", "Runners-up Grupo E", 2022],
          [55, "Winners Grupo H", "Match 56", "Runners-up Grupo G", 2022]]

# Crear un DataFrame a partir de la lista de listas
dfempty = pd.DataFrame(datavacio, columns=["", "home", "score", "away", "year"])

# Imprimir el DataFrame
print(dfempty)


                  home     score                away  year
0  48  Winners Grupo A  Match 49  Runners-up Grupo B  2022
1  49  Winners Grupo C  Match 50  Runners-up Grupo D  2022
2  50  Winners Grupo D  Match 52  Runners-up Grupo C  2022
3  51  Winners Grupo B  Match 51  Runners-up Grupo A  2022
4  52  Winners Grupo E  Match 53  Runners-up Grupo F  2022
5  53  Winners Grupo G  Match 54  Runners-up Grupo H  2022
6  54  Winners Grupo F  Match 55  Runners-up Grupo E  2022
7  55  Winners Grupo H  Match 56  Runners-up Grupo G  2022


In [50]:
df_fixture_knockout

,home,score,away,year
48,Netherlands,3–1,United States,2022
49,Argentina,2–1,Australia,2022
50,France,3–1,Poland,2022
51,England,3–0,Senegal,2022
52,Japan,1–1,Croatia,2022
53,Brazil,4–1,South Korea,2022
54,Morocco,0–0,Spain,2022
55,Portugal,6–1,Switzerland,2022


In [51]:
dict_tablas.keys()

dict_keys(['Grupo A', 'Grupo B', 'Grupo C', 'Grupo D', 'Grupo E', 'Grupo F', 'Grupo G', 'Grupo H'])

In [70]:
## actualizar el fixture de octavos con el 1 puesto (group winner) y 2 puesto (runners up)
for group in dict_tablas:
    group_winner = dict_tablas[group].loc[0, 'Team']
    runners_up = dict_tablas[group].loc[1, 'Team']
    dfempty.replace({f"Winners {group}": group_winner,
                     f"Runners-up {group}": runners_up}, inplace=True)

dfempty['winner'] = '?'
dfempty






,,home,score,away,year,winner
0,48,Netherlands,Match 49,United States,2022,?
1,49,Argentina,Match 50,Australia,2022,?
2,50,France,Match 52,Poland,2022,?
3,51,England,Match 51,Senegal,2022,?
4,52,Germany,Match 53,Morocco,2022,?
5,53,Brazil,Match 54,Uruguay,2022,?
6,54,Croatia,Match 55,Spain,2022,?
7,55,Portugal,Match 56,Switzerland,2022,?


In [73]:
dfempty = dfempty.set_index('')
dfempty

,home,score,away,year,winner
,,,,,
48,Netherlands,Match 49,United States,2022,?
49,Argentina,Match 50,Australia,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Morocco,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [94]:
## Crear funcion get_winner
def get_winner(dfempty_updated):
    for index, row in dfempty_updated.iterrows():
        home , away =  row ['home'] , row ['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away :
            winner = home
        else:
            winner = away
        dfempty_updated.loc[index, 'winner'] = winner

    return dfempty_updated

#Crear la función para calcular los puntos de cada equipo


In [95]:
dfempty

,home,score,away,year,winner
,,,,,
48,Netherlands,Match 49,United States,2022,Netherlands
49,Argentina,Match 50,Australia,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Morocco,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


# cuartos de final

In [96]:
# Define los datos del DataFrame
data10 = {
    'home': ['Winners Match 53', 'Winners Match 49', 'Winners Match 55', 'Winners Match 51'],
    'score': ['Match 58', 'Match 57', 'Match 60', 'Match 59'],
    'away': ['Winners Match 54', 'Winners Match 50', 'Winners Match 56', 'Winners Match 52'],
    'year': [2022, 2022, 2022, 2022]
}

# Crea el DataFrame con el índice específico
dfempty_quarter = pd.DataFrame(data10, index=[56, 57, 58, 59])

# Muestra el DataFrame
print(dfempty_quarter)

                home     score              away  year
56  Winners Match 53  Match 58  Winners Match 54  2022
57  Winners Match 49  Match 57  Winners Match 50  2022
58  Winners Match 55  Match 60  Winners Match 56  2022
59  Winners Match 51  Match 59  Winners Match 52  2022


In [104]:
def update_table(df_empty_round1,df_empty_round2):
    for index, row in df_empty_round1.iterrows():
        winner = df_empty_round1.loc[index , 'winner']
        match = df_empty_round1.loc[index, 'score']
        df_empty_round2.replace({f'Winners {match}':winner}, inplace= True)
        df_empty_round2['winner'] = '?'
    return df_empty_round2
    


In [105]:
update_table(dfempty,dfempty_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [106]:
get_winner(dfempty_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


creamos el dataframe para la semifinal

In [110]:
# Define los datos del DataFrame
data20 = {
    'home': ['Winners Match 57', 'Winners Match 59'],
    'score': ['Match 61', 'Match 62'],
    'away': ['Winners Match 58', 'Winners Match 60'],
    'year': [2022, 2022]
}

# Crea el DataFrame con el índice específico
dfempty_semi = pd.DataFrame(data20, index=[60, 61])

# Muestra el DataFrame
print(dfempty_semi)

                home     score              away  year
60  Winners Match 57  Match 61  Winners Match 58  2022
61  Winners Match 59  Match 62  Winners Match 60  2022


In [111]:
update_table(dfempty_quarter,dfempty_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [112]:
get_winner(dfempty_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [115]:
# Define los datos del DataFrame
data30 = {
    'home': ['Losers Match 61', 'Winners Match 61'],
    'score': ['Match 63', 'Match 64'],
    'away': ['Losers Match 62', 'Winners Match 62'],
    'year': [2022,2022]
}

# Crea el DataFrame con el índice específico
dfempty_final = pd.DataFrame(data30, index=[62,63])

# Muestra el DataFrame
print(dfempty_final)

                home     score              away  year
62   Losers Match 61  Match 63   Losers Match 62  2022
63  Winners Match 61  Match 64  Winners Match 62  2022


In [117]:
update_table(dfempty_semi,dfempty_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [118]:
get_winner(dfempty_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
